In [1]:
import pandas as pd
import pyreadr as pyreadr
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

indice_file = 1

In [2]:
import open_data
data = open_data.open()
df = data['df']

In [3]:
from ipywidgets import IntProgress
from IPython.display import display

def compute_elastic_net(df):
    regr = LogisticRegression(penalty='elasticnet' , l1_ratio=0.305, C=0.225, solver='saga', max_iter=10000)
    regr.fit(df.loc[:, df.columns != 'TARGET'].to_numpy(), df['TARGET'].to_numpy())
    return regr



def compute_bootstrap_elastic_net(df, B):
    da_ritornare = []
    f = IntProgress(min=0, max=B-1)
    display(f)
    for i in range(B):
        df_bootstrap = df.sample(frac=1, replace=True)
        da_ritornare.append(compute_elastic_net(df_bootstrap))
        f.value += 1
    return da_ritornare



def RENBOOT_pre(df, B):
    aux = compute_bootstrap_elastic_net(df, B)
    dizionario = {'intercette' : [], 'coefficienti' : []}
    for reg in aux:
        dizionario['intercette'].append(reg.intercept_)
        dizionario['coefficienti'].append(reg.coef_.flatten())
            
    dizionario['intercette'] = np.array(dizionario['intercette']).flatten()
    dizionario['coefficienti'] = np.array(dizionario['coefficienti'])

    return dizionario


In [4]:
aux = RENBOOT_pre(df, 1250)

IntProgress(value=0, max=1249)

In [5]:
np.savez('./risultati_renboot/saved_results_'+str(indice_file)+'.npz', intercette=aux['intercette'], coefficienti=aux['coefficienti'])